In [2]:
!pip install pandas numpy networkx pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.0 MB/s eta 0:00:00


# **Ploting**

# **Plot using an HTML visualization library**

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
from pyvis.network import Network

# Loading the file
df = pd.read_csv('AnswersDocument.txt', sep='\t', header=0)

# Remove empty/spaces and replace with NA
df.replace("", np.nan, inplace=True)

# Transforming the corpus into a list of lists
corpus = df.values.tolist()

# Getting the column names
nomes_colunas = df.columns

# Converting to a list
pessoas = df.columns.tolist()
print(pessoas)

# Creating a dictionary to map users to indexes
pessoa_indice = {usuario: i for i, usuario in enumerate(pessoas)}
print(pessoa_indice)

# Getting the celebrities
celebridades = []
for sublist in corpus:
    for celebridade in sublist:
        if pd.notna(celebridade):
            celebridades.append(celebridade)
celebridades = sorted(set(celebridades))

# Creating the incidence matrix
matriz_incidencia = np.zeros((len(pessoas), len(celebridades)), dtype=int)

# Assigning data to the matrix
for linha, pessoa in enumerate(pessoas):
    for coluna, celebridade in enumerate(celebridades):
        if celebridade in df[pessoa].values:
            matriz_incidencia[linha][coluna] = 1
        else:
            matriz_incidencia[linha][coluna] = 0

# Similarity matrix (inner product of the incidence matrix)
matriz_similaridade = np.dot(matriz_incidencia, matriz_incidencia.T)

# Assign 0 to the main diagonal
np.fill_diagonal(matriz_similaridade, 0)

# Converting the similarity matrix to native Python type
matriz_similaridade = matriz_similaridade.astype(int)

# Co-occurrence matrix
matriz_coocorrencia = np.dot(matriz_incidencia.T, matriz_incidencia)

# Assigning 0 to the main diagonal
np.fill_diagonal(matriz_coocorrencia, 0)

# Converting the co-occurrence matrix to native Python type
matriz_coocorrencia = matriz_coocorrencia.astype(int)

# Function to create and visualize the incidence graph
def desenha_grafo_incidencia(corpus, pessoas, celebridades):
    G_incidencia = nx.DiGraph()

    # Adding nodes for users
    for pessoa in pessoas:
        G_incidencia.add_node(pessoa, bipartite=0)

    # Adding nodes for celebrities
    for celebridade in celebridades:
        G_incidencia.add_node(celebridade, bipartite=1)  # 1 pois os nós de celebridades se conectaram ao nós dos usuários (0)

    # Adding edges between people and celebrities
    for linha, pessoa in enumerate(pessoas):
        for coluna, celebridade in enumerate(celebridades):
            if celebridade in df[pessoa].values:
                G_incidencia.add_edge(pessoa, celebridade)
    # Drawing the incidence graph
    nt = Network(notebook=True, height="750px", width="100%")
    nt.barnes_hut()
    for node in G_incidencia.nodes:
        nt.add_node(node, label=node)  # Add titles to nodes
    nt.from_nx(G_incidencia)
    nt.show("grafo_incidencia.html")

# Function to create and visualize the similarity graph
def desenha_grafo_similaridade(mtx_sim, pessoas):
    G_similaridade = nx.Graph()
    for i in range(len(pessoas)):
        for j in range(i + 1, len(pessoas)):
            if matriz_similaridade[i, j] > 0:
                G_similaridade.add_edge(pessoas[i], pessoas[j], weight=int(matriz_similaridade[i, j]))  # Converting to int

    # Drawing the similarity graph
    nt = Network(notebook=True, height="750px", width="100%")
    nt.barnes_hut()
    for node in pessoas:
        nt.add_node(node, label=node)  # Add titles to nodes
    for u, v, data in G_similaridade.edges(data=True):
        nt.add_edge(u, v, weight=data['weight'])  # Add weights to edges
    nt.from_nx(G_similaridade)
    nt.show("grafo_similaridade.html")

# Function to create and visualize the co-occurrence graph
def desenha_grafo_coocorrencia(mtx_co, celebridades):
    G_coocorrencia = nx.Graph()
    for i in range(len(celebridades)):
        for j in range(i + 1, len(celebridades)):
            if matriz_coocorrencia[i, j] > 0:
                G_coocorrencia.add_edge(celebridades[i], celebridades[j], weight=int(matriz_coocorrencia[i, j]))  # Converting to int

    # Drawing the co-occurrence graph
    nt = Network(notebook=True, height="750px", width="100%")
    nt.barnes_hut()
    for node in G_coocorrencia.nodes:
        nt.add_node(node, label=node)  # Adiciona os rótulos dos nós
    nt.from_nx(G_coocorrencia)
    nt.show("grafo_coocorrencia.html")

# Function to calculate and display the graph metrics
def metricas_grafo(grafo, nome_grafo):
    print(f"Métricas do grafo {nome_grafo}:")
    print("Número de vértices:", grafo.number_of_nodes())
    print("Número de arestas:", grafo.number_of_edges())
    graus = dict(grafo.degree())
    print("Graus de vértice:", graus)
    grau_medio = np.mean(list(graus.values()))
    print("Grau médio:", grau_medio)
    if nx.is_weighted(grafo):
        pesos = nx.get_edge_attributes(grafo, 'weight')
        peso_medio = np.mean(list(pesos.values()))
        print("Força de conectividade média:", peso_medio)
    densidade = nx.density(grafo)
    print("Densidade da rede:", densidade)

# Drawing the graphs
desenha_grafo_incidencia(corpus, pessoas, celebridades)
desenha_grafo_similaridade(matriz_similaridade, pessoas)
desenha_grafo_coocorrencia(matriz_coocorrencia, celebridades)

# Calculating and displaying the graph metrics
metricas_grafo(nx.Graph(matriz_similaridade), "de similaridade")
metricas_grafo(nx.Graph(matriz_coocorrencia), "de co-ocorrência")


['David ', 'Diego Artero', 'Felipe Lucena', 'Gabriek Yoshihiko ', 'Gustavo Moura ', 'Guilherme Ryu Mitsuzumi', 'aldair messias', 'Marcelo Vieira de Carvalho', 'Kaike Renan ', 'Felipe Rocha ', 'Gustavo gonzales', 'Elivam ', 'Gabriel Henrique de Souza Claus Rodrigues ', 'Vinicius Gatti Rodrigues ', 'Emilly Lima ', 'Paulo Henrique Fusco', 'Hugo Mercês Zampronio ', 'Davi Aguilar da Silva ', 'Bruno Bortoleto', 'Guilherme Viana Morrone ', 'filipe willian ', 'André Luís de Souza Santos ', 'Hugo', 'Gabriel Anselmo', 'Fernando Caffer', 'Giovanni Pimenta', 'Gustavo Marques', 'Gustavo Vieira ', 'Felipe Estevo', 'Allan Shinhama']
{'David ': 0, 'Diego Artero': 1, 'Felipe Lucena': 2, 'Gabriek Yoshihiko ': 3, 'Gustavo Moura ': 4, 'Guilherme Ryu Mitsuzumi': 5, 'aldair messias': 6, 'Marcelo Vieira de Carvalho': 7, 'Kaike Renan ': 8, 'Felipe Rocha ': 9, 'Gustavo gonzales': 10, 'Elivam ': 11, 'Gabriel Henrique de Souza Claus Rodrigues ': 12, 'Vinicius Gatti Rodrigues ': 13, 'Emilly Lima ': 14, 'Paulo Hen